In [1]:
%matplotlib inline

In [2]:
from __future__ import division,print_function

import os, json
import shutil
from glob import glob
import random
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
from importlib import reload
import utils; reload(utils)
from utils import plots

Using Theano backend.


In [4]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.optimizers import SGD, RMSprop, Adam


## Create validation set and sample

In [5]:
NB_ROOT = os.getcwd()

In [6]:
DATA_HOME_DIR = os.path.join(NB_ROOT, "data/invasive-species-monitoring")
results_path = os.path.join(DATA_HOME_DIR, 'results')

In [7]:
%cd $DATA_HOME_DIR
%mkdir -p valid
%mkdir -p results
%mkdir -p sample/train
%mkdir -p sample/test/unknown
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/leo/src/fastai-courses/deeplearning1/nbs/data/invasive-species-monitoring


In [8]:
def create_label_dirs(base_dir):
    """
    """
    # Create label directories thant can be recognized by Keras ImageDataGenerator.flow_from_directory    
    labels = ['invasive', 'not_invasive']
    for label in labels:
        try:
            os.makedirs(os.path.join(base_dir, label))
        except OSError as e:
            if e.errno != os.errno.EEXIST:
                raise

create_label_dirs('train')
create_label_dirs('valid')
create_label_dirs('sample/train')
create_label_dirs('sample/valid')

In [9]:
train_labels_csv = pd.read_csv("train_labels.csv")

In [10]:
def label_images_by_dir(base_dir="train"):
    """
    Move images into their label directory so that they can be recognized by
    ImageDataGenerator.flow_from_directory
    """
    for _, row in train_labels_csv.iterrows():
        image_name = "{}.jpg".format(row['name'])
        src_path = os.path.join(base_dir, image_name)
        if row['invasive'] == 1:
            dst_path = os.path.join(base_dir, 'invasive', image_name)
        else:
            dst_path = "train/not_invasive/{}".format(image_name)
        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
# label_images_by_dir()

In [11]:
def create_valid_set(train_root='train', valid_root="valid", valid_rate=0.1):
    labels = ['invasive', 'not_invasive']
    for label in labels:
        train_label_dir = os.path.join(train_root, label)
        valid_label_dir = os.path.join(valid_root, label)
        files = os.listdir(train_label_dir)
        for file in random.sample(files, k=int(len(files) * valid_rate)):
            shutil.move(os.path.join(train_label_dir, file), valid_label_dir)
#  create_valid_set()

In [12]:
def create_sample_set(rate=0.01):
    labels = ['invasive', 'not_invasive', 'unknown']
    for dataset in ['train', 'valid', 'test']:
        if dataset == "valid":
            # use a higher smple rate for the validation dataset
            # because they have fewer items
            sample_rate = rate*5
        else:
            sample_rate = rate
        for label in labels:
            src_dir = os.path.join(dataset, label)
            dst_dir = os.path.join('sample', dataset, label)
            try:
                files = os.listdir(src_dir)
            except FileNotFoundError:
                continue
            for file in random.sample(files, k=int(len(files) * sample_rate)):
                shutil.copy(os.path.join(src_dir, file), dst_dir)

# create_sample_set()

# Build Model

In [13]:
def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.

        Args: 
            x: Image array (height x width x channels)
        Returns:
            Image array (height x width x transposed_channels)
    """
    vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr


In [14]:
def addConvBlock(model, layers, filters):
    """
        Adds a specified number of ZeroPadding and Covolution layers
        to the model, and a MaxPooling layer at the very end.

        Args:
            layers (int):   The number of zero padded convolution layers
                            to be added to the model.
            filters (int):  The number of convolution filters to be 
                            created for each layer.
    """
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [15]:
def addFCBlock(model):
    """
        Adds a fully connected layer of 4096 neurons to the model with a
        Dropout of 0.5

        Args:   None
        Returns:   None
    """
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))


In [16]:
# Build Vgg model
def build_model():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))

    addConvBlock(model, 2, 64)
    addConvBlock(model, 2, 128)
    addConvBlock(model, 3, 256)
    addConvBlock(model, 3, 512)
    addConvBlock(model, 3, 512)

    model.add(Flatten())


    addFCBlock(model)
    addFCBlock(model)

    model.add(Dense(1000, activation='softmax'))

    fname = 'vgg16.h5'
    model.load_weights(get_file(fname, 'http://files.fast.ai/models/'+fname, cache_subdir='models'))
    return model
model = build_model()

In [17]:
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
        """
            Takes the path to a directory, and generates batches of augmented/normalized data. Yields batches indefinitely, in an infinite loop.

            See Keras documentation: https://keras.io/preprocessing/image/
        """
        # 224x224 is the image size used by ImageNet
        return gen.flow_from_directory(path, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [18]:
_path = DATA_HOME_DIR
# _path = DATA_HOME_DIR + '/sample' # Only for sample tests!
test_path = os.path.join(DATA_HOME_DIR, 'test')
results_path = os.path.join(DATA_HOME_DIR, 'results')
train_path = os.path.join(_path, 'train')
valid_path = os.path.join(_path, 'valid')
test_path = os.path.join(_path, 'test')

In [19]:
! cd $NB_ROOT
BATCH_SIZE=64

train_batches = get_batches(train_path, batch_size=BATCH_SIZE)
valid_batches = get_batches(valid_path, batch_size=BATCH_SIZE*2)

Found 2067 images belonging to 2 classes.
Found 228 images belonging to 2 classes.


In [20]:
def finetune(model, output_size, lr=0.001):
    # Replace the last layer with a new Dense
    model.pop()
    for layer in model.layers:
        layer.trainable=False
    model.add(Dense(output_size, activation='softmax'))
    model.compile(
        optimizer=Adam(lr=lr),
        loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
finetune(model, train_batches.nb_class)

In [22]:
model.optimizer.lr = 0.01

In [23]:
def fit_model(model):
    no_of_epochs = 3
    latest_weights_filename = None
    for epoch in range(no_of_epochs):
        print("Running epoch: %d" % epoch)
        model.fit_generator(
            train_batches, samples_per_epoch=train_batches.nb_sample,
            nb_epoch=1,
            validation_data=valid_batches, nb_val_samples=valid_batches.nb_sample
        )
        latest_weights_filename = 'ft%d.h5' % epoch
        model.save_weights(results_path+latest_weights_filename)
        print("Completed %s fit operations" % no_of_epochs)

In [24]:
fit_model(model)

Running epoch: 0
Epoch 1/1
2067/2067 [==============================] - 584s - loss: 1.5531 - acc: 0.8229 - val_loss: 0.6040 - val_acc: 0.9254
Completed 3 fit operations
Running epoch: 1
Epoch 1/1
2067/2067 [==============================] - 583s - loss: 0.9182 - acc: 0.8771 - val_loss: 0.4460 - val_acc: 0.9211
Completed 3 fit operations
Running epoch: 2
Epoch 1/1
2067/2067 [==============================] - 584s - loss: 1.0436 - acc: 0.8689 - val_loss: 0.8964 - val_acc: 0.8684
Completed 3 fit operations


In [25]:
test_batches = get_batches(
    test_path, batch_size=BATCH_SIZE * 2, shuffle=False, class_mode=None)

Found 1531 images belonging to 1 classes.


In [26]:
def predict(model, test_batches):
    return model.predict_generator(test_batches, test_batches.nb_sample)
    

In [27]:
preds = predict(model, test_batches)

In [28]:
test_files = test_batches.filenames

In [29]:
sbm = pd.DataFrame(preds, columns=["invasive","not invasive"])
sbm['name'] = [int(f.replace('unknown/', '').replace('.jpg', '')) for f in test_batches.filenames]
sbm = sbm.set_index(['name'])
sbm = sbm.sort_index()

In [30]:
sbm.to_csv('submission.csv', columns=['invasive'])

In [32]:
!kg submit -c invasive-species-monitoring submission.csv